In [4]:
# import kagglehub
# custom_path = r"C:\Users\Tanuj Rajput\OneDrive\Desktop\tanujkaggle\12"
# path = kagglehub.dataset_download("saurabhshahane/fake-news-classification")

# import shutil
# shutil.move(path, custom_path)

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("WELFake_Dataset.csv")

In [3]:
df = df.drop(columns=["Unnamed: 0"])

In [4]:
df['content'] = df['title'] + " " + df['text']

In [5]:
df.dropna(subset=['content', 'label'], inplace=True)

In [6]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [7]:
tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")
tokenizer.fit_on_texts(df['content'])

In [8]:
sequences = tokenizer.texts_to_sequences(df['content'])
padded = pad_sequences(sequences, maxlen=100, padding='post', truncating='post')

In [9]:
labels = df['label'].values

In [10]:
x_train, x_test, y_train, y_test = train_test_split(padded, labels, test_size=0.2, random_state=42)

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, Dropout

In [12]:
model = Sequential([
    Embedding(input_dim=5000, output_dim=64, input_length=100),
    LSTM(128, return_sequences=True),
    Dropout(0.3),
    LSTM(64),
    Dropout(0.3),
    Dense(32, activation='relu'),  
    Dense(1, activation='sigmoid')
])

C:\Users\Tanuj Rajput\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [13]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])   
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [15]:
history = model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test))

Epoch 1/10


1789/1789 ━━━━━━━━━━━━━━━━━━━━ 219s 122ms/step - accuracy: 0.8809 - loss: 0.3186 - val_accuracy: 0.9354 - val_loss: 0.2084
Epoch 2/10
1789/1789 ━━━━━━━━━━━━━━━━━━━━ 245s 137ms/step - accuracy: 0.9194 - loss: 0.2333 - val_accuracy: 0.8157 - val_loss: 0.3158
Epoch 3/10
1789/1789 ━━━━━━━━━━━━━━━━━━━━ 160s 89ms/step - accuracy: 0.9244 - loss: 0.2316 - val_accuracy: 0.9507 - val_loss: 0.1425
Epoch 4/10
1789/1789 ━━━━━━━━━━━━━━━━━━━━ 155s 86ms/step - accuracy: 0.9498 - loss: 0.1470 - val_accuracy: 0.9476 - val_loss: 0.1392
Epoch 5/10
1789/1789 ━━━━━━━━━━━━━━━━━━━━ 178s 99ms/step - accuracy: 0.9590 - loss: 0.1183 - val_accuracy: 0.9523 - val_loss: 0.1278
Epoch 6/10
1789/1789 ━━━━━━━━━━━━━━━━━━━━ 187s 105ms/step - accuracy: 0.9701 - loss: 0.0889 - val_accuracy: 0.9591 - val_loss: 0.1046
Epoch 7/10
1789/1789 ━━━━━━━━━━━━━━━━━━━━ 189s 106ms/step - accuracy: 0.9752 - loss: 0.0725 - val_accuracy: 0.9648 - val_loss: 0.0960
Epoch 8/10
1789/1789 ━━━━━━━━━━━━━━━━━━━━ 189s 106ms/step - accuracy: 0.9775

In [16]:
loss, accuracy = model.evaluate(x_test, y_test)
print(f"Test lossL {loss}, Test accuracy: {accuracy}")

448/448 ━━━━━━━━━━━━━━━━━━━━ 12s 26ms/step - accuracy: 0.9680 - loss: 0.1120
Test lossL 0.10683633387088776, Test accuracy: 0.9692479968070984


In [17]:
import pickle

In [19]:
model.save("fake_news_classifier_model.h5")

In [20]:
with open('tokenizer.pkl', 'wb') as f:      
    pickle.dump(tokenizer, f)
print("Model and tokenizer saved successfully.")

Model and tokenizer saved successfully.
